**Demo for `teneva.core.svd`**

---

Module contains the basic implementation of the TT-SVD algorithm as well as functions for constructing the SVD decomposition and skeleton decomposition for the matrices.

## Loading and importing modules

In [1]:
import jax
import jax.numpy as np
import teneva as teneva_base
import teneva.core_jax as teneva
from time import perf_counter as tpc
rng = jax.random.PRNGKey(42)

## Function `matrix_skeleton`

Construct truncated skeleton decomposition A = U V for the given matrix.

In [6]:
# Shape of the matrix:
m, n = 100, 30

# Build random matrix, which has rank 3,
# as a sum of rank-1 matrices:
rng, key = jax.random.split(rng)
keys = jax.random.split(key, 6)
u = [jax.random.normal(keys[i], (m, )) for i in range(3)]
v = [jax.random.normal(keys[i], (m, )) for i in range(3, 6)]
A = np.outer(u[0], v[0]) + np.outer(u[1], v[1]) + np.outer(u[2], v[2])

In [7]:
# Compute skeleton decomp.:
U, V = teneva.matrix_skeleton(A, r=3)

# Approximation error
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)

print(f'Shape of U :', U.shape)
print(f'Shape of V :', V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 3)
Shape of V : (3, 100)
Error      : 2.82e-07


In [8]:
# Compute skeleton decomp with small rank:
U, V = teneva.matrix_skeleton(A, r=2)

# Approximation error:
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)
print(f'Shape of U :', U.shape)
print(f'Shape of V :', V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 2)
Shape of V : (2, 100)
Error      : 5.11e-01


---